In [1]:
%load_ext autoreload
%autoreload 2

import os
import time
import sys

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from metadata import *
from data_manager import *
from distributed_utilities import *
from preprocess_utilities import *
from cell_utilities import *

Setting environment for AWS compute node


No vtk
Child returned 0


rm -rf /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt && mkdir -p /shared/CSHL_data_processed/MD635
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt


Child returned 1
0.38 seconds.
File does not exist: /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt
Child returned 0


rm -rf /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt && mkdir -p /shared/CSHL_data_processed/MD635
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt


Child returned 1
0.36 seconds.
File does not exist: /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt


In [2]:
# run_distributed(command='sudo apt-get install -y \
#   build-essential \
#   cython \
#   libmysqlclient-dev \
#   libhdf5-dev \
#   libxml2-dev \
#   libxslt1-dev \
#   openjdk-8-jdk \
#   python-dev \
#     python-h5py \
#     python-mysqldb \
#     python-vigra \
#   python-wxgtk3.0 \
#   python-zmq',
#                     argument_type='single',
#                    cluster_size=16)

# wait_qsub_complete()


In [3]:
# run_distributed(command='cd /shared/CellProfiler/; sudo pip install --editable .',
#                     argument_type='single',
#                    cluster_size=16)

# wait_qsub_complete()

In [4]:
download_from_s3(CELLPROFILER_PIPELINE_FP)

# Whole pipeline

In [13]:
for stack in all_nissl_stacks:
# for stack in ['MD589']:
    
    _, sections_to_filenames = DataManager.load_sorted_filenames(stack=stack)
    valid_filenames = [fn for fn in sections_to_filenames.values() if not is_invalid(fn=fn)]

    _, _, _, _, first_sec, last_sec = DataManager.load_cropbox(stack=stack)
    first_fn = sections_to_filenames[first_sec]
    last_fn = sections_to_filenames[last_sec]
    first_idx_among_valid = valid_filenames.index(first_fn)
    last_idx_among_valid = valid_filenames.index(last_fn)
    
    # Detect cells.
    
    t = time.time()
    sys.stderr.write('Detect cells using Cell Profiler ...')
    
    
    run_distributed(command='rm -r /scratch/*',
                        argument_type='single',
                       cluster_size=16)
    wait_qsub_complete()

    script_fp = os.path.join(REPO_DIR, 'cells', 'detect_cells_cellprofiler_v2.py')
    
    run_distributed(command='ROOT_DIR=/scratch/ %(script_path)s %(stack)s \'%%(filenames)s\''%\
                        {'script_path': script_fp, 
                        'stack': stack},
                        kwargs_list={'filenames': valid_filenames[first_idx_among_valid:last_idx_among_valid+1]},
                        argument_type='list2',
                       cluster_size=16)

    wait_qsub_complete()

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 1300 sec / stack

    # Generate derived data.

    t = time.time()
    sys.stderr.write('Detect cells ...')
    
    run_distributed(command='rm -r /scratch/*',
                        argument_type='single',
                       cluster_size=16)
    wait_qsub_complete()

    script_fp = os.path.join(REPO_DIR, 'cells', 'derive_cell_information.py')
    
    run_distributed(command='ROOT_DIR=/scratch/ %(script_path)s %(stack)s \'%%(filenames)s\''%\
                        {'script_path': script_fp, 
                        'stack': stack},
                        kwargs_list={'filenames': valid_filenames[first_idx_among_valid:last_idx_among_valid+1]},
                        argument_type='list2',
                       cluster_size=16)

    wait_qsub_complete()

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 700 sec / stack

Detect cells using Cell Profiler ...Child returned 0
16 nodes requested, 16 nodes available...Continuing


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
Child returned 0
16 nodes requested, 16 nodes available...Continuing


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 1122.817226 seconds
Detect cells ...Child returned 0
16 nodes requested, 16 nodes available...Continuing


...

Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 1429.613490 seconds
Detect cells ...Child returned 0
16 nodes requested, 16 nodes available...Continuing


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
Child returned 0
16 nodes requested, 16 nodes available...Continuing


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 544.575458 seconds


# Step 1: Run cellprofiler

In [ ]:
script_fp = os.path.join(REPO_DIR, 'cells', 'detect_cells_cellprofiler_v2.py')

In [ ]:
for stack in all_nissl_stacks:
# for stack in ['MD585']:
    
    _, sections_to_filenames = DataManager.load_sorted_filenames(stack=stack)
    valid_filenames = [fn for fn in sections_to_filenames.values() if not is_invalid(fn=fn)]

    _, _, _, _, first_sec, last_sec = DataManager.load_cropbox(stack=stack)
    first_fn = sections_to_filenames[first_sec]
    last_fn = sections_to_filenames[last_sec]
    first_idx_among_valid = valid_filenames.index(first_fn)
    last_idx_among_valid = valid_filenames.index(last_fn)
    
    
    run_distributed(command='rm -r /scratch/*',
                        argument_type='single',
                       cluster_size=16)
    wait_qsub_complete()
    
    
    t = time.time()
    sys.stderr.write('Detect cells using Cell Profiler ...')

    run_distributed(command='ROOT_DIR=/scratch/ %(script_path)s %(stack)s \'%%(filenames)s\''%\
                        {'script_path': script_fp, 
                        'stack': stack},
                        kwargs_list={'filenames': valid_filenames[first_idx_among_valid:last_idx_among_valid+1]},
                        argument_type='list2',
                       cluster_size=16)

    wait_qsub_complete()

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) 
    # 16 or 32 or 64 sections, 420 seconds
    # 128, 568
    # all 1300 seconds.

# Step 2: Generate derived data for 

In [8]:
script_fp = os.path.join(REPO_DIR, 'cells', 'derive_cell_information.py')

In [12]:
# for stack in all_nissl_stacks:
for stack in ['MD589']:
    
    _, sections_to_filenames = DataManager.load_sorted_filenames(stack=stack)
    valid_filenames = [fn for fn in sections_to_filenames.values() if not is_invalid(fn=fn)]

    _, _, _, _, first_sec, last_sec = DataManager.load_cropbox(stack=stack)
    first_fn = sections_to_filenames[first_sec]
    last_fn = sections_to_filenames[last_sec]
    first_idx_among_valid = valid_filenames.index(first_fn)
    last_idx_among_valid = valid_filenames.index(last_fn)
    
    
    run_distributed(command='rm -r /scratch/*',
                        argument_type='single',
                       cluster_size=16)
    wait_qsub_complete()
    
    t = time.time()
    sys.stderr.write('Detect cells ...')

    run_distributed(command='ROOT_DIR=/scratch/ %(script_path)s %(stack)s \'%%(filenames)s\''%\
                        {'script_path': script_fp, 
                        'stack': stack},
                        kwargs_list={'filenames': valid_filenames[first_idx_among_valid:last_idx_among_valid+1]},
                        argument_type='list2',
                       cluster_size=16)

    wait_qsub_complete()

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 271 sec / stack

Detect cells ...Child returned 0
16 nodes requested, 16 nodes available...Continuing


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 689.528857 seconds


In [5]:
for stack in all_nissl_stacks:
# for stack in ['MD585']:

    if stack == 'MD585': continue
    
    _, sections_to_filenames = DataManager.load_sorted_filenames(stack=stack)
    valid_filenames = [fn for fn in sections_to_filenames.values() if not is_invalid(fn=fn)]

    _, _, _, _, first_sec, last_sec = DataManager.load_cropbox(stack=stack)
    first_fn = sections_to_filenames[first_sec]
    last_fn = sections_to_filenames[last_sec]
    first_idx_among_valid = valid_filenames.index(first_fn)
    last_idx_among_valid = valid_filenames.index(last_fn)
    
    for fn in valid_filenames[first_idx_among_valid:last_idx_among_valid+1]:
        print fn
        ! aws s3 mv s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/{stack}/{fn}/{fn}_blobMaskCenters.hdf s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/{stack}/{fn}/{fn}_blobMaskCenters.bp

MD585-N16-2015.07.16-20.32.33_MD585_2_0047
move: s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-N16-2015.07.16-20.32.33_MD585_2_0047/MD585-N16-2015.07.16-20.32.33_MD585_2_0047_blobMaskCenters.hdf to s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-N16-2015.07.16-20.32.33_MD585_2_0047/MD585-N16-2015.07.16-20.32.33_MD585_2_0047_blobMaskCenters.bp
MD585-IHC16-2015.08.19-13.14.51_MD585_3_0048
move: s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-IHC16-2015.08.19-13.14.51_MD585_3_0048/MD585-IHC16-2015.08.19-13.14.51_MD585_3_0048_blobMaskCenters.hdf to s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-IHC16-2015.08.19-13.14.51_MD585_3_0048/MD585-IHC16-2015.08.19-13.14.51_MD585_3_0048_blobMaskCenters.bp
MD585-N17-2015.07.16-20.35.23_MD585_1_0049
move: s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-N17-2015.07.16-20.35.23_MD585_1_0049/MD585-N17-2015.07.16-20.35.23_MD585_1_0049_blobMaskCenters.hdf to s

move: s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-IHC20-2015.07.16-15.43.53_MD585_1_0058/MD585-IHC20-2015.07.16-15.43.53_MD585_1_0058_blobMaskCenters.hdf to s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-IHC20-2015.07.16-15.43.53_MD585_1_0058/MD585-IHC20-2015.07.16-15.43.53_MD585_1_0058_blobMaskCenters.bp
MD585-N20-2015.07.16-20.47.19_MD585_2_0059
move: s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-N20-2015.07.16-20.47.19_MD585_2_0059/MD585-N20-2015.07.16-20.47.19_MD585_2_0059_blobMaskCenters.hdf to s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-N20-2015.07.16-20.47.19_MD585_2_0059/MD585-N20-2015.07.16-20.47.19_MD585_2_0059_blobMaskCenters.bp
MD585-IHC20-2015.07.16-15.43.53_MD585_2_0059
move: s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-IHC20-2015.07.16-15.43.53_MD585_2_0059/MD585-IHC20-2015.07.16-15.43.53_MD585_2_0059_blobMaskCenters.hdf to s3://mousebrainatlas-data/CSHL_cells_v2/

MD585-N24-2015.08.19-15.29.12_MD585_1_0070
move: s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-N24-2015.08.19-15.29.12_MD585_1_0070/MD585-N24-2015.08.19-15.29.12_MD585_1_0070_blobMaskCenters.hdf to s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-N24-2015.08.19-15.29.12_MD585_1_0070/MD585-N24-2015.08.19-15.29.12_MD585_1_0070_blobMaskCenters.bp
MD585-IHC24-2015.07.18-07.14.51_MD585_1_0070
move: s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-IHC24-2015.07.18-07.14.51_MD585_1_0070/MD585-IHC24-2015.07.18-07.14.51_MD585_1_0070_blobMaskCenters.hdf to s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-IHC24-2015.07.18-07.14.51_MD585_1_0070/MD585-IHC24-2015.07.18-07.14.51_MD585_1_0070_blobMaskCenters.bp
MD585-N24-2015.08.19-15.29.12_MD585_2_0071
move: s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-N24-2015.08.19-15.29.12_MD585_2_0071/MD585-N24-2015.08.19-15.29.12_MD585_2_0071_blobMaskCenters.hdf to s

move: s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-IHC27-2015.08.19-13.28.05_MD585_2_0080/MD585-IHC27-2015.08.19-13.28.05_MD585_2_0080_blobMaskCenters.hdf to s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-IHC27-2015.08.19-13.28.05_MD585_2_0080/MD585-IHC27-2015.08.19-13.28.05_MD585_2_0080_blobMaskCenters.bp
MD585-N27-2015.08.19-15.42.30_MD585_3_0081
move: s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-N27-2015.08.19-15.42.30_MD585_3_0081/MD585-N27-2015.08.19-15.42.30_MD585_3_0081_blobMaskCenters.hdf to s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-N27-2015.08.19-15.42.30_MD585_3_0081/MD585-N27-2015.08.19-15.42.30_MD585_3_0081_blobMaskCenters.bp
MD585-IHC27-2015.08.19-13.28.05_MD585_3_0081
move: s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-IHC27-2015.08.19-13.28.05_MD585_3_0081/MD585-IHC27-2015.08.19-13.28.05_MD585_3_0081_blobMaskCenters.hdf to s3://mousebrainatlas-data/CSHL_cells_v2/

move: s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-N31-2015.08.19-16.01.47_MD585_1_0091/MD585-N31-2015.08.19-16.01.47_MD585_1_0091_blobMaskCenters.hdf to s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-N31-2015.08.19-16.01.47_MD585_1_0091/MD585-N31-2015.08.19-16.01.47_MD585_1_0091_blobMaskCenters.bp
MD585-IHC31-2015.08.19-13.48.12_MD585_1_0091
move: s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-IHC31-2015.08.19-13.48.12_MD585_1_0091/MD585-IHC31-2015.08.19-13.48.12_MD585_1_0091_blobMaskCenters.hdf to s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-IHC31-2015.08.19-13.48.12_MD585_1_0091/MD585-IHC31-2015.08.19-13.48.12_MD585_1_0091_blobMaskCenters.bp
MD585-N31-2015.08.19-16.01.47_MD585_2_0092
move: s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-N31-2015.08.19-16.01.47_MD585_2_0092/MD585-N31-2015.08.19-16.01.47_MD585_2_0092_blobMaskCenters.hdf to s3://mousebrainatlas-data/CSHL_cells_v2/dete

...

MD585-N45-2015.07.16-22.40.26_MD585_2_0134
move: s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-N45-2015.07.16-22.40.26_MD585_2_0134/MD585-N45-2015.07.16-22.40.26_MD585_2_0134_blobMaskCenters.hdf to s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-N45-2015.07.16-22.40.26_MD585_2_0134/MD585-N45-2015.07.16-22.40.26_MD585_2_0134_blobMaskCenters.bp
MD585-IHC45-2015.08.19-14.42.34_MD585_2_0134
move: s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-IHC45-2015.08.19-14.42.34_MD585_2_0134/MD585-IHC45-2015.08.19-14.42.34_MD585_2_0134_blobMaskCenters.hdf to s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-IHC45-2015.08.19-14.42.34_MD585_2_0134/MD585-IHC45-2015.08.19-14.42.34_MD585_2_0134_blobMaskCenters.bp
MD585-N45-2015.07.16-22.40.26_MD585_3_0135
move: s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-N45-2015.07.16-22.40.26_MD585_3_0135/MD585-N45-2015.07.16-22.40.26_MD585_3_0135_blobMaskCenters.hdf to s

move: s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-IHC48-2015.08.19-14.58.14_MD585_3_0144/MD585-IHC48-2015.08.19-14.58.14_MD585_3_0144_blobMaskCenters.hdf to s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-IHC48-2015.08.19-14.58.14_MD585_3_0144/MD585-IHC48-2015.08.19-14.58.14_MD585_3_0144_blobMaskCenters.bp
MD585-N49-2015.07.16-23.01.02_MD585_1_0145
move: s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-N49-2015.07.16-23.01.02_MD585_1_0145/MD585-N49-2015.07.16-23.01.02_MD585_1_0145_blobMaskCenters.hdf to s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-N49-2015.07.16-23.01.02_MD585_1_0145/MD585-N49-2015.07.16-23.01.02_MD585_1_0145_blobMaskCenters.bp
MD585-IHC49-2015.07.16-17.58.21_MD585_3_0147
move: s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-IHC49-2015.07.16-17.58.21_MD585_3_0147/MD585-IHC49-2015.07.16-17.58.21_MD585_3_0147_blobMaskCenters.hdf to s3://mousebrainatlas-data/CSHL_cells_v2/

move: s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-N52-2015.08.19-16.12.22_MD585_2_0155/MD585-N52-2015.08.19-16.12.22_MD585_2_0155_blobMaskCenters.hdf to s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-N52-2015.08.19-16.12.22_MD585_2_0155/MD585-N52-2015.08.19-16.12.22_MD585_2_0155_blobMaskCenters.bp
MD585-IHC52-2015.08.19-15.07.35_MD585_2_0155
move: s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-IHC52-2015.08.19-15.07.35_MD585_2_0155/MD585-IHC52-2015.08.19-15.07.35_MD585_2_0155_blobMaskCenters.hdf to s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-IHC52-2015.08.19-15.07.35_MD585_2_0155/MD585-IHC52-2015.08.19-15.07.35_MD585_2_0155_blobMaskCenters.bp
MD585-IHC52-2015.08.19-15.07.35_MD585_3_0156
move: s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-IHC52-2015.08.19-15.07.35_MD585_3_0156/MD585-IHC52-2015.08.19-15.07.35_MD585_3_0156_blobMaskCenters.hdf to s3://mousebrainatlas-data/CSHL_cells_v

move: s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-N56-2015.07.16-23.31.53_MD585_1_0166/MD585-N56-2015.07.16-23.31.53_MD585_1_0166_blobMaskCenters.hdf to s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-N56-2015.07.16-23.31.53_MD585_1_0166/MD585-N56-2015.07.16-23.31.53_MD585_1_0166_blobMaskCenters.bp
MD585-IHC56-2015.07.16-18.29.48_MD585_1_0166
move: s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-IHC56-2015.07.16-18.29.48_MD585_1_0166/MD585-IHC56-2015.07.16-18.29.48_MD585_1_0166_blobMaskCenters.hdf to s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-IHC56-2015.07.16-18.29.48_MD585_1_0166/MD585-IHC56-2015.07.16-18.29.48_MD585_1_0166_blobMaskCenters.bp
MD585-N56-2015.07.16-23.31.53_MD585_2_0167
move: s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-N56-2015.07.16-23.31.53_MD585_2_0167/MD585-N56-2015.07.16-23.31.53_MD585_2_0167_blobMaskCenters.hdf to s3://mousebrainatlas-data/CSHL_cells_v2/dete

MD585-N59-2015.07.16-23.44.31_MD585_1_0175
move: s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-N59-2015.07.16-23.44.31_MD585_1_0175/MD585-N59-2015.07.16-23.44.31_MD585_1_0175_blobMaskCenters.hdf to s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-N59-2015.07.16-23.44.31_MD585_1_0175/MD585-N59-2015.07.16-23.44.31_MD585_1_0175_blobMaskCenters.bp
MD585-IHC59-2015.07.16-18.42.14_MD585_3_0177
move: s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-IHC59-2015.07.16-18.42.14_MD585_3_0177/MD585-IHC59-2015.07.16-18.42.14_MD585_3_0177_blobMaskCenters.hdf to s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-IHC59-2015.07.16-18.42.14_MD585_3_0177/MD585-IHC59-2015.07.16-18.42.14_MD585_3_0177_blobMaskCenters.bp
MD585-N60-2015.07.16-23.48.44_MD585_1_0178
move: s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD585/MD585-N60-2015.07.16-23.48.44_MD585_1_0178/MD585-N60-2015.07.16-23.48.44_MD585_1_0178_blobMaskCenters.hdf to s